In [4]:
# from textblob import TextBlob
from io import StringIO
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [3]:
!pip install --upgrade numpy pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [46]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KUNAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KUNAL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
text = "I am not feeling well"

In [4]:
sentiment_score = TextBlob(text).sentiment.polarity

In [5]:
sentiment = "positive" if sentiment_score > 0 else "negative" if sentiment_score < 0 else "neutral"

In [8]:
({"sentiment": sentiment})

{'sentiment': 'neutral'}

In [28]:
with open('train.txt/train.txt', 'r') as file:
        data = file.read()

In [30]:
data_io = StringIO(data)

In [31]:
texts = []
emotions = []

for line in data_io:
    line = line.strip()  
    if line:  
        parts = line.split(';')
        if len(parts) == 2:
            texts.append(parts[0])
            emotions.append(parts[1])
        else:
            print(f"Warning: Skipping malformed line - '{line}'")

In [32]:
df = pd.DataFrame({'text': texts, 'emotion': emotions})

In [33]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [34]:
emotion_counts = df['emotion'].value_counts()
print(emotion_counts)

emotion
joy           5362
sadness       4665
anger         2159
fear          1937
love          1304
suicidal      1024
depression    1000
surprise       967
sadnessa         1
suicidalf        1
Name: count, dtype: int64


In [35]:
grouped = df.groupby('emotion')

limited_df = pd.DataFrame()

for emotion, group in grouped:
    limited_group = group.head(950)
    limited_df = pd.concat([limited_df, limited_group])

limited_df = limited_df.reset_index(drop=True)

limited_df.head()

,text,emotion
0,im grabbing a minute to post i feel greedy wrong,anger
1,i am feeling grouchy,anger
2,i think it s the easiest time of year to feel ...,anger
3,i feel irritated and rejected without anyone d...,anger
4,i already feel like i fucked up though because...,anger


In [36]:
emotion_counts = limited_df['emotion'].value_counts()
print(emotion_counts)

emotion
anger         950
depression    950
fear          950
joy           950
love          950
sadness       950
suicidal      950
surprise      950
sadnessa        1
suicidalf       1
Name: count, dtype: int64


In [58]:
emotions_to_delete = ['suicidalf', 'sadnessa']
indices_to_drop = []

for index, row in limited_df.iterrows():
    if row['emotion'] in emotions_to_delete:
        indices_to_drop.append(index)

df_cleaned = limited_df.drop(indices_to_drop)
df_cleaned = df_cleaned.reset_index(drop=True)

In [59]:
print(df_cleaned)
print(df_cleaned['emotion'].value_counts())

                                                   text   emotion
0      im grabbing a minute to post i feel greedy wrong     anger
1                                  i am feeling grouchy     anger
2     i think it s the easiest time of year to feel ...     anger
3     i feel irritated and rejected without anyone d...     anger
4     i already feel like i fucked up though because...     anger
...                                                 ...       ...
7595  I was shocked by how smooth the entire experie...  surprise
7596     I didnâ€™t think Iâ€™d be able to get this far  surprise
7597  I didnâ€™t think Iâ€™d enjoy the process this ...  surprise
7598           Iâ€™m shocked by how much Iâ€™ve learned  surprise
7599  I didnâ€™t think it would be this enjoyable, b...  surprise

[7600 rows x 2 columns]
emotion
anger         950
depression    950
fear          950
joy           950
love          950
sadness       950
suicidal      950
surprise      950
Name: count, dtype: int64


In [62]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())  # lowercase the text and tokenize it
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalnum()]
    
    # Rejoin tokens back to string
    return ' '.join(tokens)

df_cleaned['preprocessed_text'] = df_cleaned['text'].apply(preprocess_text)

In [64]:
df_cleaned.head()

,text,emotion,preprocessed_text
0,im grabbing a minute to post i feel greedy wrong,anger,im grabbing minute post feel greedy wrong
1,i am feeling grouchy,anger,feeling grouchy
2,i think it s the easiest time of year to feel ...,anger,think easiest time year feel dissatisfied
3,i feel irritated and rejected without anyone d...,anger,feel irritated rejected without anyone anythin...
4,i already feel like i fucked up though because...,anger,already feel like fucked though dont usually e...


In [82]:
X = df_cleaned['preprocessed_text']
y = df_cleaned['emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [83]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6080,) (6080,) (1520,) (1520,)


In [84]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [85]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       anger       0.82      0.78      0.80       190
  depression       0.95      1.00      0.98       190
        fear       0.75      0.79      0.77       190
         joy       0.83      0.72      0.77       190
        love       0.82      0.84      0.83       190
     sadness       0.81      0.67      0.73       190
    suicidal       0.90      1.00      0.95       190
    surprise       0.75      0.84      0.79       190

    accuracy                           0.83      1520
   macro avg       0.83      0.83      0.83      1520
weighted avg       0.83      0.83      0.83      1520



In [ ]:
def predict_emotion(text):
    processed_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([processed_text])
    return model.predict(text_tfidf)

# Test with a new entry
new_entry = ["I want to kill myself!", "I am so happy", "I dont know what to do with my life anymore", "I did not expect you here", "I feel hopeless"]
for text in new_entry:
    processed_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([processed_text])
    probs = model.predict_proba(text_tfidf)[0]
    top_2_indices = probs.argsort()[-2:]
    top_2_emotions = [model.classes_[index] for index in top_2_indices]
    top_2_probs = probs[top_2_indices]
    print(f"Top 2 emotions with probability for '{text}' are: ", {', '.join(top_2_emotions)})

Top 2 emotions with probability for 'I want to kill myself!' are:  {'fear, suicidal'}
Top 2 emotions with probability for 'I am so happy' are:  {'surprise, joy'}
Top 2 emotions with probability for 'I dont know what to do with my life anymore' are:  {'sadness, anger'}
Top 2 emotions with probability for 'I did not expect you here' are:  {'joy, surprise'}
Top 2 emotions with probability for 'I feel hopeless' are:  {'depression, suicidal'}


In [100]:
import pickle
with open('emotion_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Model saved.")

Model saved.
